# Imports

In [15]:
import random

* Make a move for X -- pick best move, train the model based on picked state and associated reward, update state
* determine best move -- based on Q grid
* Train Model -- x value is the state as an array, y value (target) is the q-value of best next move + reward at current state 
* Calc_value_of_state -- use model to predict value of state
* calc target -- is the q-value of best next move + reward at current state 
* run experiment -- runs through a tictactoe game 

In [2]:
command = 'Hello, World!'
match command:
    case 'Hello, World!':
        print('Hello to you too!')
    case 'Goodbye, World!':
        print('See you later')
    case other:
        print('No match found')

Hello to you too!


# Piece Class

In [6]:
from enum import Enum

class Piece(Enum):
    EMPTY = 'EMPTY'
    BLACK = 'BLACK'
    WHITE = 'WHITE'
    RED = ' RED '

# Board Class

In [36]:
# from piece import Piece

class Board:
    def __init__(self):
        self.pieces = [[[Piece.EMPTY for k in range(3)] for j in range(3)] for i in range(3)]
        self.winningRuns = self.getWinningRuns()

    def validMove(self,x,y,z,dir):
        if not x in range(3) or not y in range(3) or not z in range(3):
              return False
        match dir:
            case 'UP':
                return (z == 2) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y][z-1] == Piece.EMPTY or self.pieces[x][y][z-2] == Piece.EMPTY)
            case 'DOWN':
                return (z == 0) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y][z+1] == Piece.EMPTY or self.pieces[x][y][z+2] == Piece.EMPTY)
            case 'LEFT':
                return (x == 2) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x-1][y][z] == Piece.EMPTY or self.pieces[x-2][y][z] == Piece.EMPTY)
            case 'RIGHT':
                return (x == 0) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x+1][y][z] == Piece.EMPTY or self.pieces[x+2][y][z] == Piece.EMPTY)
            case 'FRONT':
                return (y == 2) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y-1][z] == Piece.EMPTY or self.pieces[x][y-2][z] == Piece.EMPTY)
            case 'BACK':
                return (y == 0) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y+1][z] == Piece.EMPTY or self.pieces[x][y+2][z] == Piece.EMPTY)
            case _:
                return False
            
    def move(self,x,y,z,dir,player: Piece):
        if not self.validMove(x,y,z,dir):
             raise ValueError
        else:
            if (self.pieces[x][y][z] == Piece.EMPTY):
                self.pieces[x][y][z] = player
            else:
                match dir:
                    case 'UP':
                        if (self.pieces[x][y][z-1] == Piece.EMPTY):
                            self.pieces[x][y][z-1] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                        else:
                            self.pieces[x][y][z-2] = self.pieces[x][y][z-1]
                            self.pieces[x][y][z-1] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                    case 'DOWN':
                        if (self.pieces[x][y][z+1] == Piece.EMPTY):
                            self.pieces[x][y][z+1] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                        else:
                            self.pieces[x][y][z+2] = self.pieces[x][y][z+1]
                            self.pieces[x][y][z+1] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                    case 'LEFT':
                        if (self.pieces[x-1][y][z] == Piece.EMPTY):
                            self.pieces[x-1][y][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                        else:
                            self.pieces[x-2][y][z] = self.pieces[x-1][y][z]
                            self.pieces[x-1][y][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                    case 'RIGHT':
                        if (self.pieces[x+1][y][z] == Piece.EMPTY):
                            self.pieces[x+1][y][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                        else:
                            self.pieces[x+2][y][z] = self.pieces[x+1][y][z]
                            self.pieces[x+1][y][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                    case 'FRONT':
                        if (self.pieces[x][y-1][z] == Piece.EMPTY):
                            self.pieces[x][y-1][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                        else:
                            self.pieces[x][y-2][z] = self.pieces[x][y-1][z]
                            self.pieces[x][y-1][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                    case 'BACK':
                        if (self.pieces[x][y+1][z] == Piece.EMPTY):
                            self.pieces[x][y+1][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player
                        else:
                            self.pieces[x][y+2][z] = self.pieces[x][y+1][z]
                            self.pieces[x][y+1][z] = self.pieces[x][y][z]
                            self.pieces[x][y][z] = player

    def getGameState(self):
        gameState = "+----------------------\n"
        gameState += "| \ " + self.pieces[0][2][0].value + "  " + self.pieces[1][2][0].value + "  " + self.pieces[2][2][0].value + " \\\n"
        gameState += "|   \                     \\\n"
        gameState += "|     \ " + self.pieces[0][1][0].value + "  " + self.pieces[1][1][0].value + "  " + self.pieces[2][1][0].value + " \\\n"
        gameState += "|       \                     \\\n"
        gameState += "|         \ " + self.pieces[0][0][0].value + "  " + self.pieces[1][0][0].value + "  " + self.pieces[2][0][0].value + " \\\n"
        gameState += "|          ---------------------|\n"
        gameState += "|   " + self.pieces[0][2][1].value + " |" + self.pieces[1][2][1].value + "  " + self.pieces[2][2][1].value + "         |\n"
        gameState += "|         |                     |\n"
        gameState += "|       " + self.pieces[0][1][1].value + "  " + self.pieces[1][1][1].value + "  " + self.pieces[2][1][1].value + "     |\n"
        gameState += "|         |                     |\n"
        gameState += "|         | " + self.pieces[0][0][1].value + "  " + self.pieces[1][0][1].value + "  " + self.pieces[2][0][1].value + " |\n"
        gameState += "|         |                     |\n"
        gameState += " \ " + self.pieces[0][2][2].value + "  " + self.pieces[1][2][2].value + "  " + self.pieces[2][2][2].value + "          |\n"
        gameState += "   \      |                     |\n"
        gameState += "     \ " + self.pieces[0][1][2].value + "  " + self.pieces[1][1][2].value + "  " + self.pieces[2][1][2].value + "      |\n"
        gameState += "       \  |                     |\n"
        gameState += "         \| " + self.pieces[0][0][2].value + "  " + self.pieces[1][0][2].value + "  " + self.pieces[2][0][2].value + " |\n"
        gameState += "           ---------------------+\n\n"
        return gameState
    

    def getWinningRuns(self):
        runs = []

        runs.append([(0,0,0),(0,0,1),(0,0,2)])
        runs.append([(0,0,0),(0,1,0),(0,2,0)])
        runs.append([(0,0,0),(1,0,0),(2,0,0)])

        runs.append([(2,2,0),(1,2,0),(0,2,0)])
        runs.append([(2,2,0),(2,1,0),(2,0,0)])
        runs.append([(2,2,0),(2,2,1),(2,2,2)])

        runs.append([(0,2,2),(0,1,2),(0,0,2)])
        runs.append([(0,2,2),(1,2,2),(2,2,2)])
        runs.append([(0,2,2),(0,2,1),(0,2,0)])

        runs.append([(2,0,2),(2,0,1),(2,0,0)])
        runs.append([(2,0,2),(1,0,2),(0,0,2)])
        runs.append([(2,0,2),(2,1,2),(2,2,2)])
        # Front
        runs.append([(0,0,0),(1,0,1),(2,0,2)])
        runs.append([(0,0,2),(1,0,1),(2,0,0)])
        runs.append([(1,0,0),(1,0,1),(1,0,2)])
        runs.append([(0,0,1),(1,0,1),(2,0,1)])
        # Top
        runs.append([(0,0,0),(1,1,0),(2,2,0)])
        runs.append([(0,2,0),(1,1,0),(2,0,0)])
        runs.append([(0,1,0),(1,1,0),(2,1,0)])
        runs.append([(1,2,0),(1,1,0),(1,0,0)])
        # Left
        runs.append([(0,0,0),(0,1,1),(0,2,2)])
        runs.append([(0,0,2),(0,1,1),(0,2,0)])
        runs.append([(0,0,1),(0,1,1),(0,2,1)])
        runs.append([(0,1,0),(0,1,1),(0,1,2)])
        # Back
        runs.append([(0,2,2),(1,2,1),(2,2,0)])
        runs.append([(0,2,0),(1,2,1),(2,2,2)])
        runs.append([(1,2,0),(1,2,1),(1,2,2)])
        runs.append([(0,2,1),(1,2,1),(2,2,1)])
        # Right
        runs.append([(2,0,2),(2,1,1),(2,2,0)])
        runs.append([(2,0,0),(2,1,1),(2,2,2)])
        runs.append([(2,0,1),(2,1,1),(2,2,1)])
        runs.append([(2,1,0),(2,1,1),(2,1,2)])
        # Bottom
        runs.append([(2,0,2),(1,1,2),(0,2,2)])
        runs.append([(0,0,2),(1,1,2),(2,2,2)])
        runs.append([(0,1,2),(1,1,2),(2,1,2)])
        runs.append([(1,0,2),(1,1,2),(1,2,2)])
        # Corners
        runs.append([(0,0,0),(1,1,1),(2,2,2)])
        runs.append([(2,0,0),(1,1,1),(0,2,2)])
        runs.append([(2,2,0),(1,1,1),(0,0,2)])
        runs.append([(0,2,0),(1,1,1),(2,0,2)])
        # Edges
        runs.append([(1,0,0),(1,1,1),(1,2,2)])
        runs.append([(2,1,0),(1,1,1),(0,1,2)])
        runs.append([(1,2,0),(1,1,1),(1,0,2)])
        runs.append([(0,1,0),(1,1,1),(2,1,2)])
        runs.append([(0,0,1),(1,1,1),(2,2,1)])
        runs.append([(2,0,1),(1,1,1),(0,2,1)])
        # Middles
        runs.append([(1,1,0),(1,1,1),(1,1,2)])
        runs.append([(1,0,1),(1,1,1),(1,2,1)])
        runs.append([(0,1,1),(1,1,1),(2,1,1)])

        return runs

    def hasWon(self,player: Piece):
        for run in self.winningRuns:
            if all(self.pieces[x][y][z] == player for (x,y,z) in run):
                return True
        return False
    
    def gameOver(self):
        return self.hasWon(Piece.RED) or self.hasWon(Piece.WHITE)

# Random Agent

In [37]:
# from board import Board
# import random

directions = ['UP','DOWN','LEFT','RIGHT','FRONT','BACK']

class RandomAgent:
    def __init__(self,player):
        self.player = player

    def getMove(self, board: Board):
        x = random.randint(0,2)
        y = random.randint(0,2)
        z = random.randint(0,2)
        dir = random.choice(directions)
        while not board.validMove(x,y,z,dir):
            x = random.randint(0,2)
            y = random.randint(0,2)
            z = random.randint(0,2)
            dir = random.choice(directions)
        return (x,y,z,dir)

# Main Class

In [19]:
# from game_player import GamePlayer

if __name__ == '__main__':
    game = GamePlayer()
    game.playGame()

+----------------------
| \ EMPTY  EMPTY  EMPTY \
|   \                     \
|     \ EMPTY  EMPTY  EMPTY \
|       \                     \
|         \ EMPTY  EMPTY  EMPTY \
|          ---------------------|
|   EMPTY |EMPTY  EMPTY         |
|         |                     |
|       EMPTY  EMPTY  EMPTY     |
|         |                     |
|         | EMPTY  EMPTY  EMPTY |
|         |                     |
 \ EMPTY  EMPTY  EMPTY          |
   \      |                     |
     \ EMPTY  EMPTY  EMPTY      |
       \  |                     |
         \| EMPTY  WHITE  EMPTY |
           ---------------------+


+----------------------
| \ EMPTY  EMPTY  EMPTY \
|   \                     \
|     \ EMPTY  EMPTY  EMPTY \
|       \                     \
|         \ EMPTY   RED   EMPTY \
|          ---------------------|
|   EMPTY |EMPTY  EMPTY         |
|         |                     |
|       EMPTY  EMPTY  EMPTY     |
|         |                     |
|         | EMPTY  EMPTY  EMPTY |
|   

# Deep Learning Agent 

## Method we might need 
* Make a move for X -- pick best move, train the model based on picked state and associated reward, update state
* determine best move -- based on Q grid
* Train Model -- x value is the state as an array, y value (target) is the q-value of best next move + reward at current state 
* Calc_value_of_state -- use model to predict value of state
* calc target -- is the q-value of best next move + reward at current state 
* run experiment -- runs through a tictactoe game 

### Notes
The value function determines how good it is to be in state s,
Agent can also learn the value of a state-action pair, which is a q value. The q funciton 
measueres tha value of choosing a particualar action when in a particular state.


Deep Q-Learning replaces the regular Q-table with a neural network. Rather than mapping a state-action pair to a q-value, a neural network maps input states to (action, Q-value) pairs.

The Bellman equation is a recursive equation that relates the value of a state to the values of its successor states. It decomposes the value function into two parts: an immediate reward and the expected discounted value of the next state. The Bellman equation is used to update the value function iteratively, until it converges to the true value function.

https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc

## Good code references:
   * https://github.com/giladariel/TicTacToe_RL/blob/master/DeepTicTacToe_org.py
   * https://github.com/mswang12/minDQN/blob/main/minDQN.py

In [41]:
def initialize_model():
    """ Initializes keras sequential model that will read in state as an array and return list of q values associated
    with each possible action. 
    """
    pass

In [51]:
class DeepLearningAgent:
    # epsilon initialized to 1 since we start random
    def __init__(self,player="White",epsilon=1, lam=1.0):
        self.player = player
        self.epsilon = epsilon
        self.lam = lam
        self.model = initialize_model()
        
    def get_reward(self, outcome):
    """ Gets the immediate reward of taking an action. Since this is tiktaktoe, if the move
    resulted in a win, reward is 1, if a loss, reward is -1, if a tie, reward is 0.5, otherwise
    the game has not ended and the reward is 0.
    """
    
    def calculate_value():
    """ The value of a specific state is predicted by the model which has been trained
    on previously visisted states.
    """
    
    def calculate_target():
    """ The target (value estimate of state s) is calculated based on the bellman equation.
    The equation combines the immediate reward from the current state and the discounted
    value of the best next state. y is discount factor which determines the balance of caring about short 
    vs long term rewards. Higher value, more weight towards long term rewards. The bellman equation is recursive.
    """
        # target (value of state s) = (reward of state s) + y * (best q value of all possible actions from state s1)
        # (best q value of all possible actions from state s1) is calculated by passing s1 into the model

    
    def train_model():
    """ The model is learning the policy that the agent will use to move around the environment
    and choose the best action. Each time the agent decides on an action, the reward (aka the 
    target) for the state (s1) is calculated. Then using s1 as x and the reward as y, the model
    undergoes one iteration of stochastic gradient descent. The output of the model is basically 
    the models prediction for the q value of the best action. 
    """
        
        # Use bellman equation (calculate_target()) to get y value
        
        # train model using model.fit
        
    def choose_best_move():
        
        
    def play_move():
        
        # initialize empty replay_memory, as no moves have been made yet
        
        # if random number is less than epsilon, do random action
        
        # else use model to predict the best move (model.predict). This will return q value which u need to find
        # associated move for 
        
        
        # add move to replay_memory. Replay_memory is a list of tuples(state,action,reward,new state)
        
        # call train model to update model
        
        # update epsilon 
        


IndentationError: expected an indented block after function definition on line 9 (3076497746.py, line 10)

# Game Player

In [43]:
# from board import Board
# from piece import Piece
# from random_agent import RandomAgent

class GamePlayer:
    def __init__(self, player1= DeepLearningAgent(Piece.WHITE), player2= RandomAgent(Piece.RED)):
        self.player1 = player1
        self.player2 = player2
        self.board = Board()

#     def playGame(self):
#         while 1 == 1:
#             # Player 1 moves
#             (x1,y1,z1,dir1) = self.player1.getMove(self.board)
#             self.board.move(x1,y1,z1,dir1,Piece.WHITE)
#             print(self.board.getGameState())
#             if self.board.hasWon(Piece.WHITE):
#                 print('White wins!')
#                 return
#             # Player 2 moves
#             (x2,y2,z2,dir2) = self.player2.getMove(self.board)
#             self.board.move(x2,y2,z2,dir2,Piece.RED)
#             print(self.board.getGameState())
#             if self.board.hasWon(Piece.RED):
#                 print('Red wins!')
#                 return

        def 